In [ ]:
! git clone https://gitlab.com/bigirqu/quranqa.git

Cloning into 'quranqa'...
remote: Enumerating objects: 333, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 333 (delta 43), reused 86 (delta 43), pack-reused 247
Receiving objects: 100% (333/333), 312.88 KiB | 12.51 MiB/s, done.
Resolving deltas: 100% (130/130), done.


# Install dependices

In [ ]:
! pip install farasa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 13.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade pip

In [ ]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00


# Import libraries

In [ ]:
!ls /content/drive/MyDrive/Quran_QA/Notebooks/

data_load_exploration.ipynb  __pycache__  quranqa22_eval.py


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd
import numpy as np
import json
import sys
from tqdm import tqdm
import time
sys.path.insert(0,"/content/drive/MyDrive/Quran_QA/Notebooks/")
import quranqa22_eval as eval

# load data

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Quran_QA/DataCsv/train.csv")
validation = pd.read_csv("/content/drive/MyDrive/Quran_QA/DataCsv/validation.csv")

In [ ]:
train_dataset_jsonl = []
with open('/content/drive/MyDrive/Quran_QA/json_data/qrcd_v1.1_train.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

with open('/content/drive/MyDrive/Quran_QA/json_data/qrcd_v1.1_dev.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

In [ ]:
len(train_dataset_jsonl)

819

# Prepare data for prediction

In [ ]:
data = pd.concat([train, validation])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 989 entries, 0 to 127
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   pq_id       989 non-null    object
 1   passage     989 non-null    object
 2   question    989 non-null    object
 3   answer      989 non-null    object
 4   start_char  989 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 46.4+ KB


In [ ]:
data = data.filter(items=["pq_id","passage","question"])
data_unique_QA = data.drop_duplicates()

In [ ]:
len(data_unique_QA)

819

# Try Different models

In [ ]:
from transformers.utils.dummy_pt_objects import RemBertForSequenceClassification
def answer_question(row):
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
    result_row = pipe(question=row["question"], context=row["passage"], handle_impossible_answer=True, topk=5)
    return result_row

In [ ]:
def Try_Transformer(tokenizer_name,model_name ):
  result = {}
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  model = AutoModelForQuestionAnswering.from_pretrained(model_name)
  # Apply the function to the "question" and "passage" columns of the DataFrame
  for i in tqdm(range(len(data_unique_QA))):
    result_part = answer_question(data_unique_QA.iloc[i])
    result.update({data_unique_QA.iloc[i]["pq_id"]:result_part})
  return result

In [ ]:
all_scores = {}

In [ ]:
tokenizer = "ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"
model = "ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 819/819 [32:04<00:00,  2.35s/it]


{'pRR': 0.3973635401364163, 'exact_match': 0.16727716727716727, 'f1': 0.36719683478146}


In [ ]:
tokenizer = "timpal0l/mdeberta-v3-base-squad2"
model = "timpal0l/mdeberta-v3-base-squad2"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 819/819 [1:13:20<00:00,  5.37s/it]


{'pRR': 0.2567396557778281, 'exact_match': 0.06105006105006105, 'f1': 0.11395546665846701}


In [ ]:
tokenizer = "wissamantoun/araelectra-base-artydiqa"
model = "wissamantoun/araelectra-base-artydiqa"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 819/819 [30:40<00:00,  2.25s/it]


{'pRR': 0.3578368997320245, 'exact_match': 0.13675213675213677, 'f1': 0.3231606589383002}


In [ ]:
tokenizer = "gfdgdfgdg/arap_qa_bert"
model = "gfdgdfgdg/arap_qa_bert"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

In [ ]:
tokenizer = "salti/AraElectra-base-finetuned-ARCD"
model = "salti/AraElectra-base-finetuned-ARCD"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

In [ ]:
tokenizer = "zohaib99k/Bert_Arabic-SQuADv2-QA"
model = "zohaib99k/Bert_Arabic-SQuADv2-QA"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

In [ ]:
tokenizer = "gfdgdfgdg/arap_qa_bert_large_v2"
model = "gfdgdfgdg/arap_qa_bert_large_v2"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

In [ ]:
tokenizer = "Damith/AraELECTRA-discriminator-QuranQA"
model = "Damith/AraELECTRA-discriminator-QuranQA"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

In [ ]:
tokenizer = "amith/AraELECTRA-discriminator-SOQAL"
model = "amith/AraELECTRA-discriminator-SOQAL"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)